In [3]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveAdultsF.p", "rb" ) )


In [4]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

,user,item,rating
0,0,0,164.0
1,0,2,3.0
2,0,7,133.0
3,0,12,2.0
4,0,15,335.0


In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [6]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>25)
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)
ratings = ratingsPD2

In [7]:
len(ratings)

5178994

In [8]:
ratings['item'].nunique()

146658

In [9]:
ratings['user'].nunique()

8711

In [10]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias

rm = 0

for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0
0.3331761712972463
0
0.34610135048491036
0
0.34042403319973097
4
0.3427298344528811
4
0.34763273548318846
Average RMSE: 0.3420128249835915


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias

for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
